<center><img src="img/dsa-logo.JPG" width="400"/>


***

<center>03 March 2026<center>
<center>Rahman Peimankar<center>


# Lecture 5 - Agenda

1. Operator Overloading and Python’s Special Methods
2. Iterators
3. Object-Oriented Programming
4. Inheritance
5. Shallow and deep copying
6. Exercises

**Quiz 1**

----- is a python special method used to initialize the values of instance members for the new object.

1) ``__self__``

2) ``__call__``

3) ``__add__``

4) ``__init__``

Please answer here: https://PollEv.com/multiple_choice_polls/6dNnE916M2hghOzBqHXdG/respond


**Quiz 2**

----- is a used to access the attributes and methods of the class in python.

1) ``__self__``

2) ``__call__``

3) ``__yield__``

4) ``__init__``

Please answer here: https://PollEv.com/multiple_choice_polls/5Wbj8rArpU2wTackPmOyx/respond

<center>
    
# 1. Operator Overloading and Python’s Special Methods

* The syntax a + b invokes addition for numeric types, yet concatenation for sequence types.
* When defining a new class, we must consider whether a syntax like a + b should be defined when a or b is an instance of that class.
* By default, the + operator is undefined for a new class.
* The author of a class may provide a definition using a technique known as **_operator overloading_**.
* This is done by implementing a specially named method.

In particular, the ``+`` operator is overloaded by implementing a method named ``__add__``, which takes the right-hand operand as a parameter and which returns the result of the expression.

The syntax, ``a + b``, is converted to a method call on object a of the form, ``a.__add__(b)``.

<center>
<img src="img/Qimage-1-1.JPG" width="700"/>

### Example: Multidimensional Vector Class

In [ ]:
print('list adding: {}'.format([5, -2, 3] + [1, 4, 2]))

However, when working with vectors, if ``u = <5,−2, 3>`` and ``v = <1, 4, 2>``, one would expect the expression, ``u + v``, to return a three-dimensional vector with coordinates ``<6, 2, 5>``.

* We therefore define a Vector class that provides a better abstraction for the notion of a geometric vector.

In [ ]:
class Vector:
    """Represent a vector in a multidimensional space."""

    def __init__(self, d):
        """Create d-dimensional vector of zeros."""
        self._coords = [0] * d

    def __len__(self):
        """Return the dimension of the vector."""
        return len(self._coords)

    def __getitem__(self, j):
        """Return jth coordinate of vector."""
        return self._coords[j]

    def __setitem__(self, j, val):
        """Set jth coordinate of vector to given value."""
        self._coords[j] = val

    def __add__(self, other):
        """Return sum of two vectors."""
        if len(self) != len(other): # relies on len method
            raise ValueError('dimensions must agree')
        result = Vector(len(self)) # start with vector of zeros
        for j in range(len(self)):
            result[j] = self[j] + other[j]
        return result

    def __eq__(self, other):
        """Return True if vector has same coordinates as other."""
        return self._coords == other._coords

    def __ne__(self, other):
        """Return True if vector differs from other."""
        return  self == other # rely on existing eq definition

    def __str__(self):
        """Produce string representation of vector."""
        return '<' + str(self._coords)[1:-1] + '>' # adapt list representation

* Internally, our vector relies upon an instance of a list, named coords, as its storage mechanism.
* By keeping the internal list encapsulated, we can enforce the desired public interface for instances of our class.
* A demonstration of supported behaviors includes the following:

In [ ]:
v = Vector(5)        # construct five-dimensional <0, 0, 0, 0, 0>
print(v)

In [ ]:
v[1] = 23            # (based on use of setitem )
print(v)

v[-1] = 45           # (also via setitem )
print(v)

In [ ]:
u = v + v           # (via add )
print(u)

In [ ]:
total = 0
for entry in v: # implicit iteration via len and getitem
    total += entry
print(total)

* We implement many of the behaviors by trivially invoking a similar behavior on the underlying list of coordinates.
* However, our implementation of ``__add__`` is customized.

### polymorphism

* It is interesting to note that the class definition, as given in class Vector, automatically supports the syntax ``u = v + [5, 3, 10, −2, 1]``.
* This results in a new vector that is the element-by-element “sum” of the first vector and the **list** instance.

In [3]:
print(v + [5, 3, 10, -2, 1])

* This is a result of Python’s polymorphism.
* Literally, **_polymorphism_** means **_many forms_**.

<center>
    
# 2. Iterators

* Iteration is an important concept in the design of data structures.
* In short, an **iterator** for a collection provides one key behavior:

    - It supports a special method named ``__next__`` that returns the next element of the collection, if any, or
    - raises a ``StopIteration`` exception to indicate that there are no further elements.

1. Fortunately, it is rare to have to directly implement an iterator class. Our preferred approach is the use of the **generator** syntax, which automatically produces an iterator of yielded values.

2. Python also helps by providing an automatic iterator implementation for any class that defines both ``__len__`` and ``__getitem__``.

To provide an instructive example of a low-level iterator, code below demonstrates just such an iterator class that works on any collection that supports both ``__len__`` and ``__getitem__``.

In [ ]:
class SequenceIterator:
    """An iterator for any of Python's sequence types."""

    def __init__(self, sequence):
        """Create an iterator for the given sequence."""
        self._seq = sequence # keep a reference to the underlying data
        self._k = -1 # will increment to 0 on first call to next

    def __next__(self):
        """Return the next element, or else raise StopIteration error."""
        self._k += 1 # advance to next index
        if self._k < len(self._seq):
            return(self._seq[self._k]) # return the data element
        else:
            raise StopIteration() # there are no more elements

    def __iter__(self):
        """By convention, an iterator must return itself as an iterator."""
        return self

### Example: Range Class

* We develop our own implementation of a class that mimics Python’s built-in ``range`` class.

In [ ]:
for k in range(1, 9, 2):
    print(k)

r = range(1, 9, 2)
print('supports __len__ method behaviour: {}'.format(len(r)))
print('supports __getitem__ method behaviour: {}'.format(r[1]))

In [ ]:
class Range:
    """A class that mimic s the built-in range class."""

    def __init__(self, start, stop=None, step=1):
        """Initialize a Range instance.

            Semantics is similar to built-in range class.
        """
        if step == 0:
            raise ValueError('step cannot be 0')

        if stop is None:             # special case of range(n)
            start, stop = 0, start   # should be treated as if range(0,n)

        # calculate the effective length once
        self._length = max(0, (stop - start + step - 1) // step)

        # need knowledge of start and step (but not stop) to support getitem
        self._start = start
        self._step = step

    def __len__(self):
        """Return number of entries in the range."""
        return self._length7

    def __getitem__(self, k):
        """Return entry at index k (using standard interpretation if negative)."""
        if k < 0:
            k += len(self) # attempt to convert negative index

        if not 0 <= k < self._length:
            raise IndexError('index out of range')

        return self._start + k * self._step

<center>
    
# 3. Object-Oriented Programming

* As the name implies, the main “actors” in the object-oriented paradigm are called **_objects_**.
* Each object is an **_instance_** of a **_class_**.
* Each class presents to the outside world a concise and consistent view of the objects that are instances of this class.

1. The class definition typically specifies **_instance variables_**, also known as **_data members_**, that the object contains.
2. It also contains the **_methods_**, known as **_member functions_**, that the object can execute.

### Object-Oriented Design Goals

<center>
<img src="img/Qimage-20.JPG" width="800"/>

### Object-Oriented Design Principles

<center>
<img src="img/Qimage-21.JPG" width="900"/>

### An Example of a Class Diagram 

In [1]:
+---------------------+      +------------------------+      +--------------------------+
|      Hospital       |      |     Department         |      |       Doctor             |
+---------------------+------|------------------------+------|--------------------------+
| - name: str         |      | - name: str            |      | - name: str              |
| - address: str      |      | - doctors: List[Doctor]|      | - specialization: str    |
| - departments:      |      | - nurses: List[Nurse]  |      | - department: Department |
|   List[Department]  |      |                        |      | - patients: List[Patient]|
+---------------------+      +------------------------+      +--------------------------+
| + add_department()  |      | + add_doctor()         |      | + schedule_appointment() |
| + get_department( ) |      | + add_nurse()          |      | + add_patient()          |
| + list_departments()|      | + get_doctor()         |      | + list_patients()        |
+---------------------+      | + get_nurse()          |      +--------------------------+
                             | + list_doctors()       | 
                             | + list_nurses()        |
                             +------------------------+

+---------------------+      +-------------------------+      +-------------------+
|       Nurse         |      |      Patient            |      |   MedicalRecord   |
+---------------------+      +-------------------------+------|-------------------+
| - name: str         |      | - name: str             |      | - record_id: int  |
| - department:       |      | - age: int              |      | - patient: Patient|
|   Department        |      | - medical_records:      |      | - doctor: Doctor  |
| - assigned_patients:|      |   List[MedicalRecord]   |      | - diagnosis: str  |
|   List[Patient]     |      +-------------------------+      | - treatment: str  |
+---------------------+      | + add_medical_record()  |      | - date: str       |
| + assign_patient()  |      | + get_medical_records() |      +-------------------+
| + list_assigned_    |      | + schedule_appointment()|      | + update_record() |
|   patients()        |      +-------------------------+      +-------------------+
+---------------------+

+---------------------+
|    Appointment      |
+---------------------+
| - patient: Patient  |
| - doctor: Doctor    |
| - date: str         |
| - status: str       |
+---------------------+
| + reschedule()      |
| + cancel()          |
| + mark_completed()  |
+---------------------+


### Class Definitions

In [12]:
# Class representing a Hospital
class Hospital:
    def __init__(self, name, address):
        self.name = name
        self.address = address
        self.departments = []

    def add_department(self, department):
        self.departments.append(department)

    def get_department(self, name):
        for department in self.departments:
            if department.name == name:
                return department
        return None

    def list_departments(self):
        return self.departments


In [13]:
# Class representing a Department within a Hospital
class Department:
    def __init__(self, name):
        self.name = name
        self.doctors = []
        self.nurses = []

    def add_doctor(self, doctor):
        self.doctors.append(doctor)

    def add_nurse(self, nurse):
        self.nurses.append(nurse)

    def get_doctor(self, name):
        for doctor in self.doctors:
            if doctor.name == name:
                return doctor
        return None

    def get_nurse(self, name):
        for nurse in self.nurses:
            if nurse.name == name:
                return nurse
        return None

    def list_doctors(self):
        return self.doctors

    def list_nurses(self):
        return self.nurses

In [ ]:
# Class representing a Doctor
class Doctor:


# Class representing a Nurse
class Nurse:
    
# Class representing a Patient
class Patient:
    
# Class representing an Appointment
class Appointment:


# Class representing a Medical Record for a Patient
class MedicalRecord:

#### The self Identifier

* In Python, the self identifier plays a key role.
* In the context of the Hospital class, there can presumably be many different Hospital instances, and each must maintain its own name, address, and department.
* Syntactically, `self` identifies the instance upon which a method is invoked.


#### The Constructor

A user can create an instance of the Hospital class using a syntax as:

In [ ]:
hospital = Hospital("OUH", "J. B. Winslows Vej 4, 5000 Odense")

### Usage Example

In [ ]:
# Create a hospital
hospital = Hospital("OUH", "J. B. Winslows Vej 4, 5000 Odense")

# Create departments
cardiology = Department("Cardiology")
orthopedics = Department("Orthopedics")

# Add departments to the hospital
hospital.add_department(cardiology)
hospital.add_department(orthopedics)

# Create doctors
dr_smith = Doctor("Dr. Smith", "Cardiologist", cardiology)
dr_jones = Doctor("Dr. Jones", "Orthopedic Surgeon", orthopedics)

# Add doctors to departments
cardiology.add_doctor(dr_smith)
orthopedics.add_doctor(dr_jones)

# Create nurses
nurse_anne = Nurse("Nurse Anne", cardiology)
nurse_bob = Nurse("Nurse Bob", orthopedics)

# Add nurses to departments
cardiology.add_nurse(nurse_anne)
orthopedics.add_nurse(nurse_bob)

# Create a patient
patient_john = Patient("John Doe", 45)

# Schedule an appointment for the patient with Dr. Smith
appointment = patient_john.schedule_appointment(dr_smith, "2024-08-10")

# Create a medical record for the patient
medical_record = MedicalRecord(1, patient_john, dr_smith, "Hypertension", "Medication", "2024-08-01")
patient_john.add_medical_record(medical_record)

# Output some information
print(f"Hospital: {hospital.name}")
print(f"Departments: {[dept.name for dept in hospital.list_departments()]}")
print(f"Dr. Smith's Patients: {[patient.name for patient in dr_smith.list_patients()]}")
print(f"John's Medical Records: {[record.diagnosis for record in patient_john.get_medical_records()]}")


<center>
    
# 4. Inheritance

* A natural way to organize various structural components of a software package is in a hierarchical fashion.

<center>
<img src="img/Qimage-3-1.JPG" width="1200"/>

* A hierarchical design is useful in software development.
* Common functionality can be grouped at the most general level, thereby promoting reuse of code.


In object-oriented programming, the mechanism for a modular and hierarchical organization is a technique known as **_inheritance_**.

* In object-oriented terminology, the existing class is typically described as the **_base class_**, **_parent class_**, or **_superclass_**, while the newly defined class is known as the **_subclass_** or **_child class_**.
* There are two ways in which a subclass can differentiate itself from its superclass:
    1. A subclass may **_specialize_** an existing behavior by providing a new implementation that **_overrides_** an existing method.
    2. A subclass may also **_extend_** its superclass by providing brand new methods.

In [ ]:
class Person:
    def __init__(self, name: str):
        self.name = name


class Doctor(Person):
    def __init__(self, name, specialization, department=None):
        super().__init__(name)
        self.specialization = specialization
        self.department = department
        self.patients = []


class Nurse(Person):
    def __init__(self, name, department=None):
        super().__init__(name)
        self.department = department
        self.assigned_patients = []


class Patient(Person):
    def __init__(self, name, age):
        super().__init__(name)
        self.age = age
        self.medical_records = []

* The mechanism for calling the inherited constructor relies on the syntax, ``super().__init__(name)``.

<center>
    
# 5. Shallow and Deep Copying

* Let's assume that ``warmtones`` denote an existing list of such colors (e.g., oranges, browns).
* We wish to create a new list named ``palette``, which is a copy of the ``warmtones`` list.
* We want to subsequently be able to add additional colors to ``palette``, or to modify or remove some of the existing colors, without affecting the contents of ``warmtones``.

``palette = warmtones``

<img src="img/Qimage-8-1.JPG" width="500"/>
    
* Unfortunately, this does not meet our desired criteria, because if we subsequently add or remove colors from “palette,” we modify the list identified as warmtones.

* We can instead create a new instance of the list class by using the syntax:
``palette = list(warmtones)``

* This is known as a **_shallow copy_**

<center>
<img src="img/Qimage-9-1.JPG" width="900"/>

* The new list is initialized so that its contents are precisely the same as the original sequence.
* This is a better situation. 
* However, if we edit a ``color`` instance from the ``palette`` list, we effectively change the contents of ``warmtones``.

* We prefer that ``palette`` be what is known as a **_deep copy_** of ``warmtones``.
* In a deep copy, the new copy references its own copies of those objects referenced by the original version.

<center>
<img src="img/Qimage-10-1.JPG" width="1100"/>

### Python’s copy Module

* Python provides a very convenient module, named ``copy``, that can produce both **_shallow copies_** and **_deep copies_** of arbitrary objects.
* This module supports two functions:
    1. The ``copy`` function creates a **_shallow copy_** of its argument
    2. the ``deepcopy`` function creates a **_deep copy_** of its argument.

``palette = copy.deepcopy(warmtones)``

<center>
    
# 6. Exercises

Please complete the code for the rest of the classes in the example above (i.e., Doctor, Nurse, Patient, Appointment, MedicalRecord).

In [ ]:
# Class representing a Doctor
class Doctor:


# Class representing a Nurse
class Nurse:
    
# Class representing a Patient
class Patient:
    
# Class representing an Appointment
class Appointment:


# Class representing a Medical Record for a Patient
class MedicalRecord:

# Thank you!
